In [ ]:
import sys

sys.path.append('..')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context('talk')

In [ ]:
from matplotlib.ticker import FuncFormatter

formatter_pct = FuncFormatter(lambda y, pos: "%.f %%" % y)
formatter_bps = FuncFormatter(lambda y, pos: "%.0f" % (10**4 * y))

In [ ]:
import dill

loaded_data = None

with open('precomputed_sims/data1.pkl', 'rb') as f:
    loaded_data = dill.load(f)

In [ ]:
maturity = loaded_data["randomization"]["maturity"]
udlyings = loaded_data["underlyings"]

In [ ]:
import numpy as np

times_cva = np.arange(0, maturity, 0.5)

shifted_times_cva = times_cva + 1.
shifted_times_cva[-1] = maturity

In [ ]:
from finance.discountfactor import ConstantRateDiscountFactor 

r = 0.02
discount = ConstantRateDiscountFactor(r)

In [ ]:
hurdle_rate = 0.1

# Code for the one year integral of loss:

\begin{equation*}
\int_{t}^{t+1} \beta_s d \varrho_s^{ccp} = 
\sum_{t < \tau_i^{\delta} \leq t+1}
\left( 
\beta_{\tau_i^{\delta}} \left( MtM^i_{\tau_i^\delta} + \Delta^i_{\tau_i^\delta} \right)
- \beta_{\tau_i} ({\rm VM}^{i}_{\tau_i} + {\rm IM}^i_{\tau_i} ) 
\right)^+ 
- \left( 
\beta_t \mathrm{CVA}_{t}^{ccp}
-\beta_{t+1} \mathrm{CVA}_{t+1}^{ccp}
\right)
\end{equation*}

In [ ]:
def compute_varrho_ccp(loss_dfr, cva_t0_dfr, cva_t1_dfr, t0_, t1_, incl_cva):
    rho = loss_dfr.sum(axis=1) 
    
    if incl_cva:
        rho += discount(t1_) * cva_t1_dfr.sum(axis=1)
        rho -= discount(t0_) * cva_t0_dfr.sum(axis=1)
    
    return rho

In [ ]:
import pandas as pd
import os

#loss_path = './res/aggregated/loss_1y_ahead/'
#cva_path = './res/aggregated/cva_ccp/'

#loss_path = './res/sim5/loss_1y_ahead/'
#cva_path = './res/sim5/cva_ccp/'

In [ ]:
def compute_1y_ahead_varrho_ccp(times_cva_, shifted_times_cva_, incl_cva=True):
    varrho_ccp = dict()
    
    for t0_, t1_ in zip(times_cva_, shifted_times_cva_):
        cur_loss_int = "[%.2f, %.2f]" % (t0_, t1_)
        loss_path_ = os.path.join(loss_path, 'loss_%s.csv' % cur_loss_int)
        loss_ahead = pd.read_csv(loss_path_, index_col=0, header=0)
    
        cur_t0 = "%.2f" % t0_
        cva_t0_path = os.path.join(cva_path, 'cva_%s.csv' % cur_t0)
        cva_t0 = pd.read_csv(cva_t0_path, index_col=0, header=0)
    
        cur_t1 = "%.2f" % t1_
        cva_t1_path = os.path.join(cva_path, 'cva_%s.csv' % cur_t1)
        cva_t1 = pd.read_csv(cva_t1_path, index_col=0, header=0)
    
        col_names = ['all', 17, 9, 29, 26, 50, 4, 5, 13, 64]
        varrho_ccp_t0_t1 = {}
    
        for col in col_names:
            if col == 'all':
                varrho_ser = compute_varrho_ccp(loss_ahead, cva_t0, cva_t1, t0_, t1_, incl_cva)
            else:
                col = unicode(col)
                loss__ = loss_ahead.drop(col, 1)
                cva_t0__ = cva_t0.drop(col, 1)
                cva_t1__ = cva_t1.drop(col, 1)
            
                varrho_ser = compute_varrho_ccp(loss__, cva_t0__, cva_t1__, t0_, t1_, incl_cva)    
    
            varrho_ccp_t0_t1[col] = varrho_ser
        
        varrho_ccp[cur_loss_int] = pd.DataFrame(varrho_ccp_t0_t1)
    
    return varrho_ccp

Code for the $KVA$:

\begin{equation*}
KVA_t = h \mathbb{E}_t \left( \int_t^T e^{-(r+h)s} DF_s ds \right)
\end{equation*}

In our context, $DF_s = DF(s)$, moreover, it is stepwise constant. So the $KVA$ definition yields:

\begin{align*}
KVA_t &=& h \left( \int_t^T e^{-(r+h)s} DF(s) ds \right) \\
&=& h \sum_{i=0}^n DF(t_i) \left( \int_{t_i}^{t_{i+1}} e^{-(r+h)s} ds \right) \\
&=& -\frac{h}{r+h} \sum_{i=0}^n DF(t_i) \left( e^{-(r+h) t_{i+1}} - e^{-(r+h) t_i} \right)
\end{align*}

with $(t_0, t_1, \dots, t_{n+1})$ the subdivision that makes jump $DF$ with $t_0 = t$ and $t_{n+1} = T$


In [ ]:
def compute_kva(r_, h_, maturity_, def_fund_serie_):
    time_grid_ = def_fund_serie_.index.values
    time_grid_ = np.append(time_grid_, maturity)
    
    exp_factors_ = np.exp(-(r_ + h_) * time_grid_)
    delta_exp_factors_ = np.ediff1d(exp_factors_)
    
    df_exp_ = np.multiply(delta_exp_factors_, def_fund_serie_.values)
    kva_ = -h_ / (r_ + h_) * df_exp_
    kva_ = kva_[::-1].cumsum()[::-1]
    
    res = pd.Series(data=kva_, index=def_fund_serie_.index)
    return res

In [ ]:
varrho_ccp = compute_1y_ahead_varrho_ccp(times_cva, shifted_times_cva, incl_cva=True)

In [ ]:
varrho_ccp_time = pd.DataFrame({key: value['all'] for (key, value) in zip(times_cva, varrho_ccp.values())})

In [ ]:
v_alpha = np.array([70, 80, 85, 90, 95, 97.5])[::-1]

In [ ]:
#f, axarr = plt.subplots(len(v_alpha), figsize = (15, 20))
for i, alpha in enumerate(v_alpha):
    var_alpha_no_time = varrho_ccp_time.quantile(alpha / 100., interpolation='linear')
    es_alpha_no_time = varrho_ccp_time[varrho_ccp_time >= var_alpha_no_time]
    es_alpha_no_time = es_alpha_no_time.mean(axis=0)
    kva_no_time = compute_kva(r, hurdle_rate, maturity, es_alpha_no_time)
    
    #axarr[i].plot(es_alpha_no_time.index, 
    plt.plot(es_alpha_no_time.index, 
                  es_alpha_no_time, 
                  label=r'$EC_t$ %.1f %%' % alpha)
    
    #axarr[i].plot(kva_no_time.index, 
    plt.plot(kva_no_time.index, 
                  kva_no_time, 
                  label=r'$KVA_t$ %.1f %%' % alpha)
    
    #axarr[i].plot(var_alpha_no_time.index, 
    plt.plot(var_alpha_no_time.index, 
                  var_alpha_no_time, 
                  label=r'$\mathbb{V}a\mathbb{R}_t$ %.1f %%' % alpha)
            
    #axarr[i].yaxis.set_major_formatter(formatter_bps)
    plt.gca().yaxis.set_major_formatter(formatter_bps)
        
    #axarr[i].legend(loc='best')
    plt.legend(loc='best')
    
    plt.savefig('./res/lin_interpol_%.2f.png' % alpha)
    plt.clf()

In [ ]:
#f, axarr = plt.subplots(len(v_alpha), figsize = (15, 20))
for i, alpha in enumerate(v_alpha):
    var_alpha_no_time = varrho_ccp_time.quantile(alpha / 100., interpolation='linear')
    es_alpha_no_time = varrho_ccp_time[varrho_ccp_time >= var_alpha_no_time]
    es_alpha_no_time = es_alpha_no_time.mean(axis=0)
    kva_no_time = compute_kva(r, hurdle_rate, maturity, es_alpha_no_time)
    
    #axarr[i].step(es_alpha_no_time.index, 
    plt.step(es_alpha_no_time.index, 
                  es_alpha_no_time, 
                  where='post', 
                  label=r'$EC_t$ %.1f %%' % alpha)
    
    #axarr[i].step(kva_no_time.index, 
    plt.step(kva_no_time.index, 
                  kva_no_time, 
                  where='post', label=r'$KVA_t$ %.1f %%' % alpha)
    
    #axarr[i].step(var_alpha_no_time.index, 
    plt.step(var_alpha_no_time.index, 
                  var_alpha_no_time, 
                  where='post', 
                  label=r'$\mathbb{V}a\mathbb{R}_t$ %.1f %%' % alpha)
            
    #axarr[i].yaxis.set_major_formatter(formatter_bps)
    plt.gca().yaxis.set_major_formatter(formatter_bps)
        
    #axarr[i].legend(loc='best')
    plt.legend(loc='best')
    
    plt.savefig('./res/step_cst_%.2f.png' % alpha)
    plt.clf()

In [ ]:
#f, axarr = plt.subplots(len(v_alpha), figsize = (15, 20))
for i, alpha in enumerate(v_alpha):
    var_alpha_no_time = varrho_ccp_time.quantile(alpha / 100., interpolation='linear')
    es_alpha_no_time = varrho_ccp_time[varrho_ccp_time >= var_alpha_no_time]
    es_alpha_no_time = es_alpha_no_time.mean(axis=0)
    kva_no_time = compute_kva(r, hurdle_rate, maturity, es_alpha_no_time)
    
    dict_to_plot = {}
    dict_to_plot[r'$KVA_t$ %.1f %%' % alpha] = kva_no_time.values
    dict_to_plot[r'$EC_t$ %.1f %%' % alpha] = es_alpha_no_time.values
    dict_to_plot[r'$\mathbb{V}a\mathbb{R}_t$ %.1f %%' % alpha] = var_alpha_no_time.values
    
    df_to_plot = pd.DataFrame(dict_to_plot, index=times_cva)
    
    #df_to_plot.plot.bar(legend=True, ax=axarr[i])   
    #axarr[i].yaxis.set_major_formatter(formatter_bps)
    
    ax = df_to_plot.plot.bar(legend=True)
    ax.yaxis.set_major_formatter(formatter_bps)
    fig = ax.get_figure()
    fig.savefig('./res/bar_%.2f.png' % alpha)

# Code to compare the default fund at time 0 between Armenti and Crepey, and the $\mathbb{E}S$ in this paper.

In [ ]:
c_ids = [17, 9, 29, 26, 50, 4, 5, 13, 64]
c_positions = [0.69, -0.46, -0.44, -0.36, 0.34, 0.23, 0.09, -0.05, -0.04]

## Code Armenti and Crepey

In [ ]:
copula = loaded_data["credit"]["copula"]
c_subsets_indexes = loaded_data["credit"]["bc_subsets_indexes"]
obligors_nb = len(copula.subsets[c_subsets_indexes[-1]][0])

positions = np.zeros(obligors_nb)
for idx, ps in zip(c_ids, c_positions):
    positions[idx] = ps

positions = positions / -positions[13]
positions = np.array(positions).reshape(positions.size, 1)

In [ ]:
udlyings = loaded_data["underlyings"]

gbm0 = udlyings[0]
kappa = gbm0.drifts[0][0]
sigma = gbm0.vols[0][0]

In [ ]:
from finance.products.european.swap import SwapContract

swap_delta = 0.25

swap_dates = SwapContract.generate_payment_dates(0, maturity, swap_delta)
swap = SwapContract(gbm0, discount, swap_dates)

In [ ]:
p_fixed = 1.
strike = swap.strike

delta_times = swap.delta_time
discount_factors = [discount(t) for t in swap.pillars[1:]]

delta_beta_sum = np.dot(delta_times, discount_factors)

notional = p_fixed / (strike * delta_beta_sum)

In [ ]:
from risk.exposures import EuropeanVaRGeomBrownianExposure

exposure = EuropeanVaRGeomBrownianExposure(swap, discount, kappa, sigma)

In [ ]:
from finance.portfolio import CCPPortfolio
from risk.basel.exposures import BaselExposureAtDefault
from risk.basel.eee import BlackScholesSwapVaREffExpectExposure
from ccp.covertworule import LCHEmirCoverTwo

port = CCPPortfolio(positions, [notional], [swap], [exposure])
eee = BlackScholesSwapVaREffExpectExposure(swap)
ead = BaselExposureAtDefault(port, [eee])

epsilon_ead = 1./12
step = 1 / 360.
delta = 5 * step
quantile_im = 0.70

exposure_at_defaults_0 = ead(t=0., epsilon=epsilon_ead, risk_period=delta, alpha=quantile_im)

cover2rule = LCHEmirCoverTwo()

default_fund_armenti_crepey_0 = cover2rule(exposure_at_defaults_0)

print "Default fund size at t=0 in Armenti and Crepey: %.2f bps" % (default_fund_armenti_crepey_0 * 10**4)

## Code $\mathbb{E}S$

In [ ]:
varrho_0 = varrho_ccp_time[0]

In [ ]:
def compute_es(alpha_, serie_):
    var_ = serie_.quantile(alpha_ / 100., interpolation='linear')
    es_ = serie_[serie_ >= var_].mean()
    return es_

def quantile_objective(alpha_, serie_, should_equal_):
    return compute_es(alpha_, serie_) - should_equal_

In [ ]:
from scipy.optimize import brentq

quantile_star = brentq(quantile_objective, 50, 100, args=(varrho_0, default_fund_armenti_crepey_0))

print "Quantile level that gives the same result: %.2f %%" % quantile_star

to_print = "Level of default fund. ES = %.3f, Armenti_Crepey = %.3f"
es_equi_df = compute_es(quantile_star, varrho_0)
print to_print % (es_equi_df * 10**4 , default_fund_armenti_crepey_0* 10**4)

In [ ]:
quantiles_level = np.linspace(60, 100, 150)
default_fund_0 = map(lambda q: compute_es(q, varrho_0), quantiles_level)

plt.plot(quantiles_level, default_fund_0, c=sns.color_palette()[0])
plt.plot(quantile_star, compute_es(quantile_star, varrho_0), 'ro', lw = 6, c=sns.color_palette()[2])
plt.annotate(r'$DF_0(a) = %.0f \, \, bps$' % (es_equi_df * 10**4) + '\n' + r'$a = %.3f \%%$' % (quantile_star), 
             xy=(quantile_star, es_equi_df), xytext=(65, 5 * es_equi_df), 
             arrowprops=dict(facecolor='black', shrink=0.02))
plt.xlabel(r'$a$')
plt.ylabel(r'$DF_0(a)$')

plt.gca().xaxis.set_major_formatter(formatter_pct)
plt.gca().yaxis.set_major_formatter(formatter_bps)

plt.savefig('./res/df_equi_a.png')

#plt.show()

In [ ]:
var_alpha_no_time = varrho_ccp_time.quantile(quantile_star / 100., interpolation='linear')
es_alpha_no_time = varrho_ccp_time[varrho_ccp_time >= var_alpha_no_time]
es_alpha_no_time = es_alpha_no_time.mean(axis=0)
kva_alpha_no_time = compute_kva(r, hurdle_rate, maturity, es_alpha_no_time)

plt.plot(es_alpha_no_time.index,
         es_alpha_no_time,
         label=r'$EC_t$ %.2f %%' % quantile_star)

plt.plot(kva_alpha_no_time.index,
         kva_alpha_no_time,
         label=r'$KVA_t$ %.2f %%' % quantile_star)

plt.plot(var_alpha_no_time.index,
         var_alpha_no_time,
         label=r'$\mathbb{V}a\mathbb{R}_t$ %.2f %%' % quantile_star)

#es_alpha_no_time.plot(legend=True, 
#                      label=r'$EC_t$ %.2f %%' % quantile_star)

#kva_alpha_no_time.plot(legend=True, 
#                       label=r'$KVA_t$ %.2f %%' % quantile_star)

#var_alpha_no_time.plot(legend=True, 
#                       label=r'$\mathbb{V}a\mathbb{R}_t$ %.2f %%' % quantile_star)

plt.gca().yaxis.set_major_formatter(formatter_bps)
plt.legend(loc='best')
 
plt.savefig('./res/lin_interpol_optimal.png')
    
#plt.show()

In [ ]:
plt.step(es_alpha_no_time.index, 
         es_alpha_no_time, 
         where='post', 
         label=r'$\mathbb{E}S_t$ %.2f %%' % quantile_star)

plt.step(kva_alpha_no_time.index, 
         kva_alpha_no_time, 
         where='post', label=r'$KVA_t$ %.2f %%' % quantile_star)

plt.step(var_alpha_no_time.index, 
         var_alpha_no_time, 
         where='post', 
         label=r'$\mathbb{V}a\mathbb{R}_t$ %.2f %%' % quantile_star)

plt.gca().yaxis.set_major_formatter(formatter_bps)
plt.legend(loc='best')

plt.savefig('./res/step_cst_optimal.png')

#plt.show()

In [ ]:
dict_to_plot = {}
dict_to_plot[r'$KVA_t$ %.2f %%' % quantile_star] = kva_alpha_no_time.values
dict_to_plot[r'$EC_t$ %.2f %%' % quantile_star] = es_alpha_no_time.values
dict_to_plot[r'$\mathbb{V}a\mathbb{R}_t$ %.2f %%' % quantile_star] = var_alpha_no_time.values
    
df_to_plot = pd.DataFrame(dict_to_plot, index=times_cva)
df_to_plot.plot.bar(legend=True, xticks=times_cva)

plt.gca().yaxis.set_major_formatter(formatter_bps)
plt.legend(loc='best')

plt.savefig('./res/bar_optimal.png')

#plt.show()

# Computations of $X^{(-j)}$

## Code computation for IM in Armenti and Crepey

In [ ]:
def armenti_crepey_im_0(c_id_):
    return port.compute_exposure(0., risk_period=delta, conf_level=quantile_im, towards_=c_id_)
    
armenti_crepey_im_0_ids = map(armenti_crepey_im_0, c_ids)

im_props = armenti_crepey_im_0_ids / sum(armenti_crepey_im_0_ids) * 100

im_props = pd.Series(im_props.flatten(), index=map(unicode, c_ids))

print im_props

## Code for our paper

In [ ]:
print varrho_ccp.keys()

In [ ]:
varrho_ccp_0 = varrho_ccp['[0.00, 1.00]']

var_0 = varrho_ccp_0.quantile(quantile_star / 100., interpolation='linear')
es_0 = varrho_ccp_0[varrho_ccp_0 >= var_0].mean(axis=0)

delta_es = es_0['all'] - es_0

print "Delta ES / sum (Delta ES)"
print
print delta_es / delta_es.sum() * 100

In [ ]:
col_names = ['all', 17, 9, 29, 26, 50, 4, 5, 13, 64]

kva_serie_0 = pd.Series(index=map(unicode, col_names))

for col_name in col_names:
    c_id = unicode(col_name)
    varrho_ccp_time_c_id = pd.DataFrame({key: value[c_id] for (key, value) in zip(times_cva, varrho_ccp.values())})
    
    var_alpha_no_time_c_id = varrho_ccp_time_c_id.quantile(quantile_star / 100., interpolation='linear')
    es_alpha_no_time_c_id = varrho_ccp_time_c_id[varrho_ccp_time_c_id >= var_alpha_no_time_c_id].mean(axis=0)
    
    kva_no_time_c_id = compute_kva(r, hurdle_rate, maturity, es_alpha_no_time_c_id)
    
    kva_c_id_0 = kva_no_time_c_id[0]
    
    kva_serie_0[c_id] = kva_c_id_0
    
delta_kva = kva_serie_0['all'] - kva_serie_0

print "Delta KVA / sum (Delta KVA)"
print
print delta_kva / delta_kva.sum() * 100

In [ ]:
es_prop = delta_es / delta_es.sum() * 100
es_prop = es_prop.drop('all')

kva_prop = delta_kva / delta_kva.sum() * 100
kva_prop = kva_prop.drop('all')

bc_ids = [17, 9, 29, 26, 50, 4, 5, 13, 64]
spreads = [176, 45, 367, 1053, 73, 56, 52, 61, 108]
ids_spreads = dict((id_, spread_) for (id_, spread_) in zip(bc_ids, spreads))

sorted_bc_ids = sorted(ids_spreads.items(), key=lambda x: x[1])
sorted_bc_ids = [x[0] for x in sorted_bc_ids]

def map_id_label(c_id_):
    pos_ = positions[c_id_]
    spread_ =  ids_spreads[c_id_]
    
    pos_string_ = "%.2f" % abs(pos_)
    if pos_ < 0:
        pos_string_ = "(" + pos_string_ + ")"
    
    return '%i bps \n%s' % (spread_, pos_string_) 

labels = map(map_id_label, sorted_bc_ids)
print labels

In [ ]:
proportions_df = pd.DataFrame(columns=[r'$\mu_0^{EC}$', r'$\mu_0^{KVA}$', r'$\mu_0^{IM}$'], index=labels)

for id_ in sorted_bc_ids:
    label_id_ = map_id_label(id_)

    es_prop_id = es_prop[unicode(id_)]
    proportions_df.set_value(label_id_, r'$\mu_0^{EC}$', es_prop_id)
    
    kva_prop_id = kva_prop[unicode(id_)]
    proportions_df.set_value(label_id_, r'$\mu_0^{KVA}$', kva_prop_id)
    
    im_prop_id = im_props[unicode(id_)]
    proportions_df.set_value(label_id_, r'$\mu_0^{IM}$', im_prop_id)
    
ax = proportions_df.plot.bar()

ax.yaxis.set_major_formatter(formatter_pct)
plt.xticks(rotation=0)

plt.savefig('./res/proportions.png')

plt.show()